In [72]:
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd


In [51]:
def is_number(s):
    """ Returns True is string is a number. """
    return s.replace('.','',1).isdigit()

In [89]:
data_dict = {}
source = BeautifulSoup(open("./data/download/AAA.html"), "html.parser")

ticker_code = "AAA"
# Parsing date
days = [datetime.strptime(x.get_text().strip(), "%Y-%m-%d")  for x in source.select("li div.row-time.noline")[1:]]
data_dict["ticker_code"] = ["AAA" for x in range(len(days))]
data_dict["date"] = days


# Parsing prices
prices = [(float(x.get_text().strip()) if is_number(x.get_text().strip()) else x.get_text().strip()) for x in source.select("li div.row1")]
data_dict["open"] = prices[6::6]
data_dict["highest"] = prices[7::6]
data_dict["lowest"] = prices[8::6]
data_dict["close"] = prices[9::6]
data_dict["average"] = prices[10::6]
data_dict["adjusted"] = prices[11::6]

# Parsing volume
volumes = [((float(x.get_text().strip())) if is_number(x.get_text().strip()) else None) for x in source.select("li div.row3")[2:]]
data_dict["trading_volume"] = volumes[0::2]
data_dict["put_through_volume"] = volumes[1::2]

df = pd.DataFrame(data_dict)
df.head(10)
df.to_csv(f"{ticker_code}.csv",index = None)